# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9a80b2cb80>
├── 'a' --> tensor([[-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595]])
└── 'x' --> <FastTreeValue 0x7f9a80aa1250>
    └── 'c' --> tensor([[ 0.0957,  0.3080,  0.9720, -0.2256],
                        [-1.2331,  0.6387, -0.6974,  1.3196],
                        [ 1.5165,  1.5576, -0.0351, -0.3848]])

In [4]:
t.a

tensor([[-0.1791,  1.2245, -0.5387],
        [ 1.9300, -0.6600,  1.1595]])

In [5]:
%timeit t.a

73.7 ns ± 0.799 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9a80b2cb80>
├── 'a' --> tensor([[-1.4951, -1.1420, -0.1572],
│                   [-0.2155, -1.2206, -1.4992]])
└── 'x' --> <FastTreeValue 0x7f9a80aa1250>
    └── 'c' --> tensor([[ 0.0957,  0.3080,  0.9720, -0.2256],
                        [-1.2331,  0.6387, -0.6974,  1.3196],
                        [ 1.5165,  1.5576, -0.0351, -0.3848]])

In [7]:
%timeit t.a = new_value

77.8 ns ± 1.5 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1791,  1.2245, -0.5387],
               [ 1.9300, -0.6600,  1.1595]]),
    x: Batch(
           c: tensor([[ 0.0957,  0.3080,  0.9720, -0.2256],
                      [-1.2331,  0.6387, -0.6974,  1.3196],
                      [ 1.5165,  1.5576, -0.0351, -0.3848]]),
       ),
)

In [10]:
b.a

tensor([[-0.1791,  1.2245, -0.5387],
        [ 1.9300, -0.6600,  1.1595]])

In [11]:
%timeit b.a

69.3 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9010, -0.4782, -1.6182],
               [ 0.8076,  2.0138,  0.7661]]),
    x: Batch(
           c: tensor([[ 0.0957,  0.3080,  0.9720, -0.2256],
                      [-1.2331,  0.6387, -0.6974,  1.3196],
                      [ 1.5165,  1.5576, -0.0351, -0.3848]]),
       ),
)

In [13]:
%timeit b.a = new_value

633 ns ± 5.65 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.02 µs ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.3 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

285 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

277 µs ± 4.66 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f99f46ad040>
├── 'a' --> tensor([[[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]],
│           
│                   [[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]],
│           
│                   [[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]],
│           
│                   [[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]],
│           
│                   [[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]],
│           
│                   [[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]],
│           
│                   [[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]],
│           
│                   [[-0.1791,  1.2245, -0.5387],
│                    [ 1.9300, -0.6600,  1.1595]]])
└── 'x' --> <FastTreeValue 0x7f9aae477700>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.5 µs ± 429 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9aac406be0>
├── 'a' --> tensor([[-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595],
│                   [-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595],
│                   [-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595],
│                   [-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595],
│                   [-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595],
│                   [-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595],
│                   [-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595],
│                   [-0.1791,  1.2245, -0.5387],
│                   [ 1.9300, -0.6600,  1.1595]])
└── 'x' --> <FastTreeValue 0x7f9aae472f40>
    └── 'c' --> tensor([[ 0.0957,  0.3080,  0.9720, -0.2256],
                        [-1.2331,  0.6387, -0.6974,  1.3196],
                 

In [23]:
%timeit t_cat(trees)

41.6 µs ± 487 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79.5 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0957,  0.3080,  0.9720, -0.2256],
                       [-1.2331,  0.6387, -0.6974,  1.3196],
                       [ 1.5165,  1.5576, -0.0351, -0.3848]],
              
                      [[ 0.0957,  0.3080,  0.9720, -0.2256],
                       [-1.2331,  0.6387, -0.6974,  1.3196],
                       [ 1.5165,  1.5576, -0.0351, -0.3848]],
              
                      [[ 0.0957,  0.3080,  0.9720, -0.2256],
                       [-1.2331,  0.6387, -0.6974,  1.3196],
                       [ 1.5165,  1.5576, -0.0351, -0.3848]],
              
                      [[ 0.0957,  0.3080,  0.9720, -0.2256],
                       [-1.2331,  0.6387, -0.6974,  1.3196],
                       [ 1.5165,  1.5576, -0.0351, -0.3848]],
              
                      [[ 0.0957,  0.3080,  0.9720, -0.2256],
                       [-1.2331,  0.6387, -0.6974,  1.3196],
                       [ 1.5165,  1.5576, -0.0351, -0.3848]],

In [26]:
%timeit Batch.stack(batches)

154 µs ± 889 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0957,  0.3080,  0.9720, -0.2256],
                      [-1.2331,  0.6387, -0.6974,  1.3196],
                      [ 1.5165,  1.5576, -0.0351, -0.3848],
                      [ 0.0957,  0.3080,  0.9720, -0.2256],
                      [-1.2331,  0.6387, -0.6974,  1.3196],
                      [ 1.5165,  1.5576, -0.0351, -0.3848],
                      [ 0.0957,  0.3080,  0.9720, -0.2256],
                      [-1.2331,  0.6387, -0.6974,  1.3196],
                      [ 1.5165,  1.5576, -0.0351, -0.3848],
                      [ 0.0957,  0.3080,  0.9720, -0.2256],
                      [-1.2331,  0.6387, -0.6974,  1.3196],
                      [ 1.5165,  1.5576, -0.0351, -0.3848],
                      [ 0.0957,  0.3080,  0.9720, -0.2256],
                      [-1.2331,  0.6387, -0.6974,  1.3196],
                      [ 1.5165,  1.5576, -0.0351, -0.3848],
                      [ 0.0957,  0.3080,  0.9720, -0.2256],
                   

In [28]:
%timeit Batch.cat(batches)

241 µs ± 3.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

626 µs ± 29.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
